# Identification of modal parametes using extended Morlet-Wave method  from MDOF system
ver. 0.1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from morlet_wave import *

**Steps required by the user:**
1. Load Impulse Response Functions as a numpy array of the shape: \
`x[(number_of_samples, measure_points)]`
2. Define sampling frequency: \
eg. `fs = 1024` S/s
3. Estimate natural frequencies: \
eg. `nat_freq_est = np.array([315, 860, 1667]) * 2*np.pi` \
unit of natural frequencies is [rad/s]. In case of noisy signals it is important to estimate natural frequency as accurate as it is possible.

In [2]:
# x = 
# fs = 
# nat_freq_est =

**Set parameters of the method:**
1. Set time spread parameters. One can set any two set parameters, but according to the author these three sets should be used:
    * `tsprd = (5, 10)`
    * `tsprd = (7, 14)` <- default
    * `tsprd = (10, 20)`
2. Set range of morlet-wave function cycles, default value:
    * `ncycl = (30, 300)` <- default

In [3]:
tsprd = (5, 10)
ncycl = (30, 300)

Defined container to store identified modal parameters:

In [4]:
data = {
    "zeta" : [],
    "omega": [],
    "X"    : []
}

Following cell does the identification. It iterates along all measurement spots and natural frquencies and stores data in container `data`.\
*Note:* in case of very noisy data, if identified natural frequencies varies significantly from estimated, then estimated natural frequencies can be used for identification, but calling method for frequency identification: ```detect_frequency(use_estimated=True)``` 

In [ ]:
measure_points = x.shape[1]
nat_freq = nat_freq_est.size
for i in range(measure_points):
    zeta = []
    omega = []
    X = []
    for j in range(nat_freq):
        print("i, j: ", i, j)
        if j == 0:
            freq = (nat_freq_est[0], nat_freq_est[1])
        elif j == nat_freq-1:
            freq = (nat_freq_est[-1], nat_freq_est[-2])
        elif np.abs(nat_freq_est[j]-nat_freq_est[j+1]) < np.abs(nat_freq_est[j]-nat_freq_est[j-1]):
            freq = (nat_freq_est[j], nat_freq_est[j+1])
        else:
            freq = (nat_freq_est[j], nat_freq_est[j-1])
            
        sys = ExtendedMW(fs, x[i,], freq, tsprd, ncycl)
        sys.detect_frequency()
        sys.detect_damp()
        sys.estimate(True)
        sys.detect_amplitude(True)
        
        zeta.append(sys.zeta)
        omega.append(sys.omega)
        X.append(sys.X * np.exp(1j * sys.phi))
        del sys
    data["zeta"].append(zeta)
    data["omega"].append(omega)
    data["X"].append(X)

Calculate modeshapes from the identified amplitudes and phases.

In [ ]:
psi = np.sign(np.sin(np.angle(beam["X"])))*np.abs(beam["X"])
print(psi)

Plot the mode shapes:

In [ ]:
m = np.linspace(1, measure_points, measure_points)
for i in range(nat_freq):
    y = np.zeros(measure_points)
    m = np.linspace(1, measure_points, measure_points)
    m[np.isnan(psi[:, i])] = np.nan
    y[np.invert(np.isnan(psi[:, i]))] = psi[np.invert(np.isnan(psi[:, i])), i]
    print(m, y)
    plt.plot(m, y/np.max(np.abs(y)))
plt.grid(True)
plt.xticks(np.linspace(1, measure_points, measure_points));